# Udacity - Machine Learning Engineer Nanodegree
## Capstone Project
### Title: Development of a LSTM Network to Predict Students’ Answers on Exam Questions

### Implementation of DKT:
#### Part 1: Define constants

In [1]:
dataset = "data/ASSISTments_skill_builder_data.csv" # Dataset path
best_model_file = "saved_models/ASSISTments.best.model.weights.hdf5" # File to save the model.
train_log = "logs/dktmodel.train.log" # File to save the training log.
eval_log = "logs/dktmodel.eval.log" # File to save the testing log.
optimizer = "adagrad" # Optimizer to use
lstm_units = 250 # Number of LSTM units
batch_size = 20 # Batch size
epochs = 10 # Number of epochs to train
dropout_rate = 0.6 # Dropout rate
verbose = 1 # Verbose = {0,1,2}
testing_rate = 0.2 # Portion of data to be used for testing
validation_rate = 0.2 # Portion of training data to be used for validation

#### Part 2: Pre-processing

In [2]:
from Utils import *

dataset, num_skills = read_file(dataset)
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(dataset, validation_rate, testing_rate)

print("======== Data Summary ========")
print("Data size: %d" % len(dataset))
print("Training data size: %d" % len(X_train))
print("Validation data size: %d" % len(X_val))
print("Testing data size: %d" % len(X_test))
print("Number of skills: %d" % num_skills)
print("==============================")

======== Data Summary ========
Data size: 4163
Training data size: 2665
Validation data size: 666
Testing data size: 832
Number of skills: 123


#### Part 3: Building the model

In [3]:
from StudentModel import DKTModel, DataGenerator

# Create generators for training/testing/validation
# Remove the slice ([0:10]) to train with the whole dataset
train_gen = DataGenerator(X_train[0:10], y_train[0:10], num_skills, batch_size)
val_gen = DataGenerator(X_val[0:10], y_val[0:10], num_skills, batch_size)
test_gen = DataGenerator(X_test[0:10], y_test[0:10], num_skills, batch_size)

# Create model
student_model = DKTModel(num_skills=train_gen.num_skills,
                      num_features=train_gen.feature_dim,
                      optimizer=optimizer,
                      hidden_units=lstm_units,
                      batch_size=batch_size,
                      dropout_rate=dropout_rate)

/home/d2/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Part 4: Train the Model

In [4]:
history = student_model.fit(train_gen,
                  epochs=epochs,
                  val_gen=val_gen,
                  verbose=verbose,
                  filepath_bestmodel=best_model_file,
                  filepath_log=train_log)

==== Training Started ====
Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 0.6934 - val_loss: 0.6733

Epoch 00001: val_loss improved from inf to 0.67332, saving model to saved_models/ASSISTments.best.model.weights.hdf5
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 0.6172 - val_loss: 0.8462 - val_auc: 0.4906 - val_acc: 0.6176 - val_pre: 0.6221

Epoch 00002: val_loss did not improve from 0.67332
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.7396 - val_loss: 0.6766 - val_auc: 0.4374 - val_acc: 0.5435 - val_pre: 0.6015

Epoch 00003: val_loss did not improve from 0.67332
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 0.5638 - val_loss: 0.7032 - val_auc: 0.5558 - val_acc: 0.6038 - val_pre: 0.6171

Epoch 00004: val_loss did not improve from 0.67332
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 0.5792 - val_loss: 0.8531 - val_auc: 0.4701 - val_acc: 0.5650 - val_pre: 0.6119

Epoch 0

#### Part 5: Load the Model with the Best Validation Loss

In [5]:
student_model.load_weights(best_model_file)

#### Part 6: Test the Model

In [6]:
result = student_model.evaluate(test_gen, metrics=['auc','acc','pre'], verbose=verbose, filepath_log=eval_log)

==== Evaluation Started ====
1/1 [==============================] - 7s 7s/step - auc: 0.5795 - acc: 0.4913 - pre: 0.4826
==== Evaluation Done ====
